In [1]:
import pandas as pd
import time
import torch

## Загрузка данных

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [3]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [6]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

## Делаем массив с данными

In [7]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [8]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [9]:
X[0:10]

tensor([[18,  9, 27, 27,  8, 15, 10,  7, 22, 22, 13, 10, 12, 26,  9, 16,  1, 10,
         16, 26,  9, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7, 15, 15,  0, 18, 23,  6, 10,  7, 15, 15,  0, 18, 23,  6,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 3, 15, 15,  0, 17, 22, 22, 10,  3, 15, 15,  0, 17, 22, 22,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 8, 18, 10, 16,  6,  2,  8, 19, 27, 10, 16, 22, 10, 16, 15,  9,  5, 26,
         10, 18,  9, 27, 27,  8, 15, 10, 16, 26,  9, 16, 10, 19,  9, 16, 23,  6,
         15, 10, 25, 22, 15,  1, 19, 16, 10, 15, 19, 25, 10, 12],
        [ 8, 16,  1, 10,  7,  8, 13, 15, 10,  9, 19, 10, 22, 20, 10, 22, 19,  7,
       

## Смотрим на Embedding и RNN ячейку

In [10]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:10])
t

tensor([[[ 0.9004,  0.0259,  0.4117,  ...,  0.6157, -1.0859, -2.1504],
         [-0.9102,  1.0716,  1.6357,  ..., -0.2527, -0.7056, -1.3963],
         [-0.1736, -0.3153, -2.1766,  ..., -0.0573,  1.9306, -0.6378],
         ...,
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662],
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662],
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662]],

        [[-1.4930, -1.8454,  0.8021,  ...,  0.9483, -0.2257,  1.3879],
         [ 0.1923,  1.1125, -1.9005,  ..., -0.0716, -0.0777, -1.6635],
         [ 0.1923,  1.1125, -1.9005,  ..., -0.0716, -0.0777, -1.6635],
         ...,
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662],
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662],
         [ 0.3864, -1.1453,  1.5496,  ...,  0.0854, -0.2570, -0.7662]],

        [[ 0.1963, -2.4907,  0.6245,  ...,  0.3238, -0.1116,  1.7483],
         [ 0.1923,  1.1125, -1.9005,  ..., -0

In [11]:
t.shape, X[0:10].shape

(torch.Size([10, 50, 28]), torch.Size([10, 50]))

In [12]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

In [13]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

## Практика. Реализуйте код модели нейронной сети
3 слоя - embeding (28), скрытая ячейка (128), полносвязанный из состояния rnn в букву (28)

In [15]:
class Network(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        out, _ = self.rnn(embed)
        return self.linear(out)

In [16]:
vocab_size = len(CHAR_TO_INDEX)
embed_dim = 28
hidden_dim = 128

model = Network(vocab_size, embed_dim, hidden_dim)

In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [19]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    rnn.train()
    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 4.323, Train loss: 2.085
Epoch 1. Time: 4.170, Train loss: 1.816
Epoch 2. Time: 4.764, Train loss: 1.737
Epoch 3. Time: 6.380, Train loss: 1.688
Epoch 4. Time: 6.189, Train loss: 1.654
Epoch 5. Time: 4.715, Train loss: 1.628
Epoch 6. Time: 4.109, Train loss: 1.607
Epoch 7. Time: 4.223, Train loss: 1.588
Epoch 8. Time: 4.186, Train loss: 1.572
Epoch 9. Time: 4.121, Train loss: 1.558


## Практика. Реализуйте код генерации следующей буквы на основе модели
Логика такая:
    - Сначала кормим в нее буквы из sentence (прогревая состояние)
    - Затем пока не получим none (0) берем самую вероятную букву и добавляем ее в sentence
    - Повторяем

In [34]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    X = torch.zeros((1, len(sentence)), dtype=int)
    for j, w in enumerate(sentence):
      if j >= MAX_LEN:
        break
      X[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    for i in range(MAX_LEN):
      o = model(X)
      w = torch.argmax(o[-1, -1,:], keepdim=True)
      X = torch.cat([X, w.unsqueeze(0).unsqueeze(0)], axis=1)
      char = INDEX_TO_CHAR[w]
      if char == "none":
        break
      sentence.append(char)
    return "".join(sentence)

In [27]:
generate_sentence()

'hello the the the the the the the the the the the the t'

In [35]:
for ep in range(50):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    print(generate_sentence())

Epoch 0. Time: 4.247, Train loss: 1.228
hellore it the some on the some on the some on the some
Epoch 1. Time: 4.182, Train loss: 1.226
hello it the some on the some on the some on the some o
Epoch 2. Time: 4.171, Train loss: 1.225
hello it the some on the some on the some on the some o
Epoch 3. Time: 4.205, Train loss: 1.223
hello it the some on the some on the some on the some o
Epoch 4. Time: 4.157, Train loss: 1.221
hello it the some on the some on the some on the some o
Epoch 5. Time: 4.158, Train loss: 1.219
hello it the some on the some on the some on the some o
Epoch 6. Time: 4.135, Train loss: 1.218
hello it the some on the some on the some on the some o
Epoch 7. Time: 4.147, Train loss: 1.216
hello it the some on the some on the some on the some o
Epoch 8. Time: 4.138, Train loss: 1.214
hello it the some on the some on the some on the some o
Epoch 9. Time: 4.159, Train loss: 1.213
hello it the simplet it the simplet it the simplet it t
Epoch 10. Time: 4.255, Train loss: 1.211